### KDEF dataset preprocessing 

#### Original dataset folder structure: 
The original dataset is organised in the following way.<br>
Example folder structure : KDEF/AM02/AM02SUFL.JPG 

AM02SUFL.JPG <br>
**Letter 1**: Session <br>
A = series one <br>
B = series two <br>

**Letter 2**: Gender<br>
F = female<br>
M = male<br>

**Letter 3 & 4**: Identity number<br>
01 - 35<br>

**Letter 5 & 6**: Expression<br>
AF = afraid<br>
AN = angry<br>
DI = disgusted<br>
HA = happy<br>
NE = neutral<br>
SA = sad<br>
SU = surprised<br>

**Letter 7 & 8**: Angle<br>
FL = full left profile<br>
HL = half left profile<br>
S = straight<br>
HR = half right profile<br>
FR = full right profile

#### Folder structure needed:
We need to group the pictures by its emotion class. <br>

**KDEF_aug**/<br>
Afraid<br>
Angry<br>
Disgusted<br>
Happy<br>
Neutral<br>
Sad<br>
Surprised<br>

Also, we need to convert the images to a square shape of constant dimensions(326x326) so that its compatible with the InceptionV3 model.

In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpim
from shutil import copyfile
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
%matplotlib inline

In [5]:
kdef_path = './KDEF'
kdef_aug_path = './KDEF_aug'

emotions = {
    "AF": "Afraid",
    "AN": "Angry",
    "DI": "Disgusted",
    "HA": "Happy",
    "NE": "Neutral",
    "SA": "Sad",
    "SU": "Surprised"
}

In [12]:
# Creating a folder for each class
for emotion in list(emotions.values()):
    os.mkdir(os.path.join(kdef_aug_path,emotion))

In [13]:
# Crop the image to square and resize it to a 326x326 size image
def crop_and_resize(image_path):
    image = imageio.imread(image_path)

    crop = iaa.CropToSquare()
    croped_image = crop.augment_image(image)
            
    resize = iaa.Resize(326)
    resized_image = resize.augment_image(croped_image)

    return resized_image

# Augmentation by adding Gaussian noise to increase the dataset size
def add_gaussian_noise(image):
    gaussian_noise=iaa.AdditiveGaussianNoise(10,30)
    noise_image=gaussian_noise.augment_image(image)

    return noise_image

**Note**: Then names of the images **AF31V, AM31H** did not comply with the naming scheme of the dataset.<br>
So changed their names to the following, after manually observing images with similar emotion. <br>
AF31V -> AF31SA<br>
AM31H -> AF31SU<br>

In [ ]:
for f in os.listdir(kdef_path):
    folder_path = kdef_path + '/' + f

    for image_name in os.listdir(folder_path):
        image_path = folder_path + '/' + image_name

        try:
            e = emotions[image_name[4:6]] # Get emotion class from image name

            resized_image = crop_and_resize(image_path)
            target_path = kdef_aug_path + '/' + e + '/' + image_name # Construct the path to the new folder
            imageio.imsave(target_path, resized_image)

            noise_image = add_gaussian_noise(resized_image)
            noise_image_name = image_name[:-4] + '_gaus.JPG'
            target_path = kdef_aug_path + '/' + e + '/' + noise_image_name # Construct the path to the new folder
            imageio.imsave(target_path, noise_image_name)

        except Exception as ex:
            print(f"Exception : {image_path} : {ex} ")
            continue

The dataset has been converted to the desired structure.<br>
Have also doubled the size of the dataset by adding Gaussian noise to the resized images.<br>

Now we can precoeed with the traning.